In [1]:
#Import

from pymatgen.analysis.interfaces.substrate_analyzer import SubstrateAnalyzer
from pymatgen.analysis.interfaces.coherent_interfaces import CoherentInterfaceBuilder
from pymatgen.analysis.interfaces.zsl import ZSLGenerator
from pymatgen.core.structure import Structure

from pymatgen.core.surface import get_symmetrically_distinct_miller_indices
from pymatgen.core.surface import SlabGenerator

from pymatgen.ext.matproj import MPRester
api_key = "kJhjnOu7tx7q2ddENmIhMexGuOujnGcV"


import crystal_toolkit
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt


import os
from ase.optimize import FIRE
from ase.io import read, write
from ase.atoms import Atoms
from sevenn.calculator import SevenNetCalculator


# Conversion factor
ev_per_a2_to_j_per_m2 = 16.0217657

/home/anton/python3.10_env/notebookenv_3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Li anode coatings

### Get final list of suitable compounds

In [45]:
#paths to data tables with suitable coatings and barriers
path_table_coatings = 'data/Li_anode_coatings_relaxed.csv'
path_table_barriers = 'data/Li_percolation_barriers_MACE.csv'
path_table_sol = 'data/LLZO_dop4 - LLZO_dop4.csv'

table_coatings = pd.read_csv(path_table_coatings)
table_barriers = pd.read_csv(path_table_barriers)
table_sol = pd.read_csv(path_table_sol)

In [54]:
# set criteria for barrier limit
e_m_lim = 0.6 #eV
e_sol_lim = 0.510 #eV

In [62]:
# get final list of compounds

list_of_compounds = []
list_of_compounds2 = []
list_of_compounds_final = []

for index, row in table_barriers.iterrows():
    e_m = round(row['e3d'],2)
    mat_id = row['material_id']
    if 0 < e_m < e_m_lim: 
        # print(f"Row {index}: Em 3d = {e_m}, material_id = {mat_id}")
        list_of_compounds.append(mat_id)
        
for index, row in table_sol.iterrows():
    if row['d_energy_dft'] != 'No':
        e_sol = round(float(row['d_energy_dft']),3)
        mat_id = row['name_maxr'].split('_')[1]
        if e_sol < e_sol_lim: 
            # print(f"Row {index}: Em 3d = {e_m}, material_id = {mat_id}")
            list_of_compounds2.append(mat_id)

list_of_compounds_sol_mig = (set(list_of_compounds)&set(list_of_compounds2))
        
for index, row in table_coatings.iterrows():
    mat_id = row['material_id']
    if mat_id in list_of_compounds_sol_mig:
        formula = row['formula_pretty']
        # print(f'Compound {formula}')
        list_of_compounds_final.append(mat_id)

# print(list_of_compounds,list_of_compounds2)
new_df = table_coatings[table_coatings["material_id"].isin(list_of_compounds_sol_mig)]
merged_df = pd.merge(new_df, table_barriers, on="material_id", how="inner")
df_sorted = merged_df.sort_values(by="e3d")
df_sorted.index = range(1, len(df_sorted) + 1)

# Reordering columns
df_sorted2 = df_sorted[[
    "formula_pretty", "space_group", "crystal_system", "nsites",
    "e1d", "e2d", "e3d", "fmax", "material_id", "theoretical",
    "reduction_limit", "oxidation_limit", "reduction_reaction",
    "oxidation_reaction", "chemsys", 
    "energy_above_hull", "band_gap", "is_stable"
]]

# Rounding e1d, e2d, and e3d to 2 decimal places
df_sorted2[["e1d", "e2d", "e3d"]] = df_sorted2[["e1d", "e2d", "e3d"]].round(2)

In [63]:
df_sorted2

,formula_pretty,space_group,crystal_system,nsites,e1d,e2d,e3d,fmax,material_id,theoretical,reduction_limit,oxidation_limit,reduction_reaction,oxidation_reaction,chemsys,energy_above_hull,band_gap,is_stable
1,Li2HN,Pnma,Orthorhombic,16,0.09,0.12,0.12,0.089242,mp-1189725,True,0.25,0.66,4 Li2HN -> 4 Li2HN,4 Li2HN -> 2 LiH2N + 0.6667 LiN3 + 5.333 Li,H-Li-N,0.00,2.08,True
2,Li3GaN2,Ia-3,Cubic,48,0.15,0.15,0.15,0.086796,mp-3887,False,0.13,0.77,8 Li3GaN2 -> 8 Li3GaN2,8 Li3GaN2 -> 8 GaN + 2.667 LiN3 + 21.33 Li,Ga-Li-N,0.00,2.38,True
3,Li3AlN2,Ia-3,Cubic,48,0.20,0.20,0.20,0.096252,mp-13944,False,0.00,0.79,8 Li3AlN2 -> 8 Li3AlN2,8 Li3AlN2 -> 8 AlN + 2.667 LiN3 + 21.33 Li,Al-Li-N,0.00,2.94,True
4,Sr2LiCBr3N2,Fd-3m,Cubic,36,0.28,0.28,0.28,0.078636,mp-569782,False,0.00,2.14,4 Sr2LiCBr3N2 -> 4 Sr2LiCBr3N2,4 Sr2LiCBr3N2 -> 2 SrCN2 + 6 SrBr2 + 2 N2 + 2 ...,Br-C-Li-N-Sr,0.00,3.97,True
5,LiGdO2,I4_1/amd,Tetragonal,8,0.37,0.37,0.37,0.070564,mp-754204,True,0.00,2.93,2 LiGdO2 -> 2 LiGdO2,2 LiGdO2 -> 0.5 Li2O2 + Gd2O3 + Li,Gd-Li-O,0.04,2.88,False
6,LiBr,P6_3mc,Hexagonal,4,0.40,0.40,0.40,0.067395,mp-976280,True,0.00,3.67,2 LiBr -> 2 LiBr,2 LiBr -> 2 Br + 2 Li,Br-Li,0.00,4.94,True
7,LiI,P6_3mc,Hexagonal,4,0.46,0.46,0.46,0.077815,mp-570935,False,0.00,2.84,2 LiI -> 2 LiI,2 LiI -> 2 I + 2 Li,I-Li,0.00,4.38,True
8,LiH,Fm-3m,Cubic,2,0.47,0.47,0.47,0.072075,mp-23703,False,0.00,0.99,LiH -> LiH,LiH -> 0.5 H2 + Li,H-Li,0.00,2.98,True
9,Li2Te,Fm-3m,Cubic,3,0.49,0.49,0.49,0.076824,mp-2530,False,0.00,1.56,Li2Te -> Li2Te,Li2Te -> 0.3333 LiTe3 + 1.667 Li,Li-Te,0.00,2.49,True


In [64]:
list_of_compounds_final

['mp-3887',
 'mp-13944',
 'mp-23703',
 'mp-2530',
 'mp-976280',
 'mp-754204',
 'mp-1189725',
 'mp-570935',
 'mp-569782']

### Get data from MP

In [65]:
# Create a MPRester object with the API key
dic_st_final = {}
for matproj_id in list_of_compounds_final:

    with MPRester(api_key) as mpr:
        compounds = mpr.materials.summary.search(material_ids=[matproj_id],  fields = [
                                            'structure',
                                            'material_id',
                                            'symmetry',
                                            'theoretical'
                                            ])

    # mpr = MPRester(api_key)
    # ws = mpr.get_wulff_shape("mp-985585")
    compound = compounds[0]
    st = compound.structure
    os.makedirs(f"interfaces_with_Li/{st.reduced_formula}/",exist_ok=True)
    dic_st_final[matproj_id] = st

Retrieving SummaryDoc documents: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 12336.19it/s]


In [103]:
# st = dic_st_final['mp-3887']
# st.add_oxidation_state_by_guess().as_dict()

In [96]:
# st.sites[40].specie.oxi_state
# site.specie.symbol, 0) for site in surface_atoms

-3.0

# Create interfaces

In [105]:
import numpy as np
from pymatgen.core import Structure
from pymatgen.analysis.interfaces.zsl import ZSLGenerator
from pymatgen.analysis.interfaces.coherent_interfaces import CoherentInterfaceBuilder
from pymatgen.analysis.interfaces.substrate_analyzer import SubstrateAnalyzer


def matches(substrate_bulk, film_bulk, substrate_miller =None, film_max_miller =4, misfit = 5):
    # Find matches between fixed substrate and film with misfit criterion
    out_list = []
    # out_dic = {'substrate_hkl':None, 'film_hkl':None, 'misfit':None}
    
    sub_analyzer = SubstrateAnalyzer(film_max_miller =film_max_miller, bidirectional = False)
    sub_analyzer.calculate(film=film_bulk,substrate=substrate_bulk)
    matches = list(sub_analyzer.calculate(film=film_bulk,substrate=substrate_bulk, substrate_millers=[substrate_miller]))


    filtered_matches = []
    film_millers = []
    # Process each match
    for match in matches:
        film_matrix = match.film_transformation
        substrate_matrix = match.substrate_transformation

        # Extract original in-plane lattice vectors from bulk film
        original_vectors_s = np.array([substrate_bulk.lattice.matrix[0], 
                                     substrate_bulk.lattice.matrix[1]])

        # Apply transformation matrix to get new film lattice vectors
        new_vectors_s = np.dot(substrate_matrix, original_vectors_s)

        # Compute misfit (strain) in x and y directions
        misfit_x_s = round(abs((np.linalg.norm(new_vectors_s[0]) - np.linalg.norm(original_vectors_s[0])) / np.linalg.norm(original_vectors_s[0])),1)
        misfit_y_s = round(abs((np.linalg.norm(new_vectors_s[1]) - np.linalg.norm(original_vectors_s[1])) / np.linalg.norm(original_vectors_s[1])),1)
        
        
        # Extract original in-plane lattice vectors from bulk film
        original_vectors = np.array([film_bulk.lattice.matrix[0], 
                                     film_bulk.lattice.matrix[1]])

        # Apply transformation matrix to get new film lattice vectors
        new_vectors = np.dot(film_matrix, original_vectors)

        # Compute misfit (strain) in x and y directions
        misfit_x = round(abs((np.linalg.norm(new_vectors[0]) - np.linalg.norm(original_vectors[0])) / np.linalg.norm(original_vectors[0])),1)
        misfit_y = round(abs((np.linalg.norm(new_vectors[1]) - np.linalg.norm(original_vectors[1])) / np.linalg.norm(original_vectors[1])),1)

        # Apply filtering conditions
        if misfit_x <= misfit and misfit_y <= misfit:
            filtered_matches.append(match)
            
            if match.film_miller not in film_millers:
                film_millers.append(match.film_miller)
                
                out_list.append([substrate_miller, match.film_miller, [round(misfit_x,1), round(misfit_y,1)], match.von_mises_strain, [round(misfit_x_s,1),round(misfit_y_s,1)]])

                print(f"Film miller: {match.film_miller}")
                print(f"Match area: {match.match_area:.4f}")
                print(f"Von_mises_strain: {match.von_mises_strain:.4f}")
                print(f"Film Misfit along x: {misfit_x:.4f}")
                print(f"Film Misfit along y: {misfit_y:.4f}")
                print(f"Sub Misfit along x: {misfit_x_s:.4f}")
                print(f"Sub Misfit along y: {misfit_y_s:.4f}\n\n")
                
    
    return(out_list)


def compute_surface_density(structure, select="top", layer_thickness=1.0):
    a_vector = structure.lattice.matrix[0]
    b_vector = structure.lattice.matrix[1]
    surface_area = np.linalg.norm(np.cross(a_vector, b_vector))
    cartesian_z_coords = np.array([site.coords[2] for site in structure])
    z_max = np.max(cartesian_z_coords)
    z_min = np.min(cartesian_z_coords)
    
    if select == "top":
        surface_atoms = [site for site in structure if (z_max - site.coords[2] <= layer_thickness)]
    elif select == "bottom":
        surface_atoms = [site for site in structure if (site.coords[2] - z_min <= layer_thickness)]
    else:
        raise ValueError("Invalid selection! Use 'top' or 'bottom'.")
    
    num_surface_atoms = len(surface_atoms)
    return num_surface_atoms / surface_area



def compute_surface_charge_density(structure, select="top", layer_thickness=1.0):
    
    oxidation_states = {'O': -2, 'Li': +1, 'Cl': -1, 'F': -1, 'N' : -3, 'B': +3, 'I':  -1, 'Be':+2, 'Ga':+3,
                       'Sc': +3, 'Al': +3, 'H' : -1, 'S': -2, 'Br': -1, 'Te':-2, 'Tm': +3, 'Gd':+3, 'Sr':+2, 
                        'C': +4, 'Zr': +4, 'La': +3, 'P': +5, 'Si': +4}
    a_vector = structure.lattice.matrix[0]
    b_vector = structure.lattice.matrix[1]
    surface_area = np.linalg.norm(np.cross(a_vector, b_vector))
    cartesian_z_coords = np.array([site.coords[2] for site in structure])
    z_max = np.max(cartesian_z_coords)
    z_min = np.min(cartesian_z_coords)
    
    if select == "top":
        surface_atoms = [site for site in structure if (z_max - site.coords[2] <= layer_thickness)]
    elif select == "bottom":
        surface_atoms = [site for site in structure if (site.coords[2] - z_min <= layer_thickness)]
    else:
        raise ValueError("Invalid selection! Use 'top' or 'bottom'.")
    
    # surface_charge = sum(oxidation_states.get(site.specie.symbol, 0) for site in surface_atoms)
    # surface_charge = sum(oxidation_states.get(site.specie.symbol, 0) for site in surface_atoms)
     # Use oxidation state from site.specie
    try:
        surface_charge = sum(site.specie.oxi_state for site in surface_atoms)
    except AttributeError as e:
        raise ValueError("Structure must have oxidation states assigned to each site. You can use `Structure.add_oxidation_state_by_guess()` or provide them manually.") from e
    return surface_charge / surface_area

def create_interfaces(substrate_bulk, film_bulk, substrate_miller, film_miller, film_max_miller=4, num_sites_limit = 200, 
                      density_limit = 0.1, charge_limit = 0.1, gap=2.0, vacuum_over_film=15.0, film_thickness=5, substrate_thickness=7,
                     surface_thickness = 0.8, misfit = 5, match = None):
    i = -1
    all_interfaces = []
    dic_list = []
    sub_formula = substrate_bulk.composition.reduced_formula
    film_formula = film_bulk.composition.reduced_formula
    
    zsl = ZSLGenerator(max_area=200, max_area_ratio_tol=0.05, max_length_tol=0.05, max_angle_tol=1, bidirectional=False)
    seen_interfaces = set()
    
    cib = CoherentInterfaceBuilder(film_structure=film_bulk, substrate_structure=substrate_bulk, 
                                   film_miller=film_miller, substrate_miller=substrate_miller, 
                                   zslgen=zsl, filter_out_sym_slabs = True)
    
    terminations = cib.terminations


    for termination in terminations:
        interfaces = list(cib.get_interfaces(termination=termination, gap=gap, vacuum_over_film=vacuum_over_film,
                            film_thickness=film_thickness, substrate_thickness=substrate_thickness, in_layers=False))

        for interface in interfaces:
            # interface_id = (interface.num_sites, termination)
            # Check if substrate lattice was altered
            original_substrate_lattice = substrate_bulk.lattice.matrix
            new_substrate_lattice = interface.substrate.lattice.matrix

                   
                
            interface_id = (termination)
            if interface.num_sites < num_sites_limit and interface_id not in seen_interfaces:
                dic = {'substrate':sub_formula, 'material_id': m[5],
                       'film':film_formula, 
                       'hkl_sub':match[0], 'hkl_film':match[1], 
                       'misfit_x': match[2][0], 'misfit_y': match[2][1], 
                       'misfit_x_sub': match[4][0], 'misfit_y_sub': match[4][1], 
                       'von_mises':round(match[3],3), 'termination': termination, 'n_at': interface.num_sites, 
                       'slab': None, 'substrate_density': None, 'film_density': None, 
                       'substrate_charge_density': None, 'film_charge_density': None,
                       'abs_charge_density':None}
                
                
                if interface.num_sites < num_sites_limit:
                    i += 1
                    all_interfaces.append(interface)
                    seen_interfaces.add(interface_id)

                    substrate_density = compute_surface_density(interface.substrate, select="top", layer_thickness = surface_thickness)
                    film_density = compute_surface_density(interface.film, select="bottom", layer_thickness = surface_thickness)

                    substrate_charge_density = compute_surface_charge_density(interface.substrate, select="top", layer_thickness = surface_thickness)
                    film_charge_density = compute_surface_charge_density(interface.film, select="bottom", layer_thickness = surface_thickness)

                    total_charge_density = abs(substrate_charge_density + film_charge_density)



                    if substrate_density > density_limit and film_density > density_limit and total_charge_density < charge_limit:

                        dic['substrate_density'] = round(substrate_density,3)
                        dic['film_density'] = round(film_density,3)
                        dic['substrate_charge_density'] = round(substrate_charge_density,3)
                        dic['film_charge_density'] = round(film_charge_density,3)
                        dic['abs_charge_density'] = round(film_charge_density,3)

                        t1 = termination[0].replace('/', '')
                        t2 = termination[1].replace('/', '')
                        filename = f'{sub_formula}_{film_formula}_{"".join(map(str, substrate_miller))}_{"".join(map(str, film_miller))}_{interface.num_sites}at_{t1}_{t2}'
                        dic['slab'] = filename
                        interface.to(filename=f'interfaces_with_Li/{sub_formula}/{filename}.POSCAR', fmt="poscar")
                        dic_list.append(dic)

    return dic_list, all_interfaces

In [110]:
#Necessary parameters

# substrate_bulk = st
film_bulk = Structure.from_file("Li.cif")
film_bulk.add_oxidation_state_by_element({'Li': +1})

substrate_millers = [(0,0,1),(1,1,0), (1,1,1)]  # Specify the Miller index of the substrate
film_max_miller = 1  # Max Miller index for the film
num_sites_limit = 200  # Limit for total atoms in the interface
misfit = 5  # Max misfit percentage allowed
density_limit = 0.05
charge_limit = 0.1
film_thickness=5
substrate_thickness=7
surface_thickness = 0.5


final_interfaces = []

film_miller_list = [(0, 0, 1), (1, 1, 0), (1, 1, 1)]  # Example set of Miller indices for the film

for mp in dic_st_final.keys():
    substrate_bulk = dic_st_final[mp].add_oxidation_state_by_guess()
    print(mp, substrate_bulk.composition.reduced_formula)
    substrate = substrate_bulk.composition.reduced_formula
    film = film_bulk.composition.reduced_formula

    for substrate_miller in substrate_millers:
        for substrate_miller in substrate_millers:
            matches_i = matches(substrate_bulk, film_bulk, substrate_miller, film_max_miller =film_max_miller, misfit = misfit)
            for m in matches_i:
                film_miller = m[1]
                # print(film_miller)
                m.append(mp)
                # print(m)
                dic_list, interfaces = create_interfaces(
                        substrate_bulk, film_bulk, substrate_miller, film_miller, film_max_miller=film_max_miller,
                        num_sites_limit = num_sites_limit, density_limit = density_limit, charge_limit = charge_limit,
                        film_thickness=5, substrate_thickness=7, surface_thickness = 0.8, misfit = misfit, match = m
                    )
            
                
                final_interfaces.extend(dic_list)


mp-3887 Li3GaN2
Film miller: (1, 1, 0)
Match area: 66.9142
Von_mises_strain: 0.0108
Film Misfit along x: 1.2000
Film Misfit along y: 1.0000
Sub Misfit along x: 0.0000
Sub Misfit along y: 0.0000


Film miller: (1, 1, 0)
Match area: 334.5710
Von_mises_strain: 0.0224
Film Misfit along x: 3.0000
Film Misfit along y: 4.0000
Sub Misfit along x: 2.0000
Sub Misfit along y: 0.0000


Film miller: (1, 1, 0)
Match area: 66.9142
Von_mises_strain: 0.0108
Film Misfit along x: 1.2000
Film Misfit along y: 1.0000
Sub Misfit along x: 0.0000
Sub Misfit along y: 0.0000


Film miller: (1, 1, 0)
Match area: 334.5710
Von_mises_strain: 0.0224
Film Misfit along x: 3.0000
Film Misfit along y: 4.0000
Sub Misfit along x: 2.0000
Sub Misfit along y: 0.0000


Film miller: (1, 1, 0)
Match area: 66.9142
Von_mises_strain: 0.0108
Film Misfit along x: 1.2000
Film Misfit along y: 1.0000
Sub Misfit along x: 0.0000
Sub Misfit along y: 0.0000


Film miller: (1, 1, 0)
Match area: 334.5710
Von_mises_strain: 0.0224
Film Misfit a

In [111]:
import pandas as pd
df = pd.DataFrame(final_interfaces)
# print(df[['termination', 'n_at', 'slab', 'substrate_density', 'film_density', 'substrate_charge_density', 'film_charge_density']])
df.to_csv("interface_with_Li_summary.csv")

In [112]:
df


,substrate,material_id,film,hkl_sub,hkl_film,misfit_x,misfit_y,misfit_x_sub,misfit_y_sub,von_mises,termination,n_at,slab,substrate_density,film_density,substrate_charge_density,film_charge_density,abs_charge_density
0,Li3GaN2,mp-3887,Li,"(1, 1, 0)","(1, 1, 0)",3.0,4.0,2.0,0.0,0.022,"(Li_Cmmm_1, Li2N_P-1_12)",135,Li3GaN2_Li_110_110_135at_Li_Cmmm_1_Li2N_P-1_12,0.107,0.116,-0.036,0.116,0.116
1,Li3GaN2,mp-3887,Li,"(1, 1, 0)","(1, 1, 0)",3.0,4.0,2.0,0.0,0.022,"(Li_Cmmm_1, Li2N_P-1_12)",135,Li3GaN2_Li_110_110_135at_Li_Cmmm_1_Li2N_P-1_12,0.107,0.116,-0.036,0.116,0.116
2,Li3GaN2,mp-3887,Li,"(1, 1, 0)","(1, 1, 0)",3.0,4.0,2.0,0.0,0.022,"(Li_Cmmm_1, Li2N_P-1_12)",135,Li3GaN2_Li_110_110_135at_Li_Cmmm_1_Li2N_P-1_12,0.107,0.116,-0.036,0.116,0.116
3,LiH,mp-23703,Li,"(1, 1, 0)","(1, 0, 0)",2.0,2.0,4.6,12.0,0.005,"(Li_P4/mmm_1, LiH_P4/mmm_2)",128,LiH_Li_110_100_128at_Li_P4mmm_1_LiH_P4mmm_2,0.248,0.083,0.000,0.083,0.083
4,LiH,mp-23703,Li,"(1, 1, 0)","(1, 0, 0)",2.0,2.0,4.6,12.0,0.005,"(Li_P4/mmm_1, LiH_P4/mmm_2)",128,LiH_Li_110_100_128at_Li_P4mmm_1_LiH_P4mmm_2,0.248,0.083,0.000,0.083,0.083
5,LiH,mp-23703,Li,"(1, 1, 0)","(1, 0, 0)",2.0,2.0,4.6,12.0,0.005,"(Li_P4/mmm_1, LiH_P4/mmm_2)",128,LiH_Li_110_100_128at_Li_P4mmm_1_LiH_P4mmm_2,0.248,0.083,0.000,0.083,0.083
6,Li2Te,mp-2530,Li,"(0, 0, 1)","(1, 1, 0)",3.0,4.0,2.0,5.0,0.012,"(Li_Cmmm_1, Te_R-3m_1)",51,Li2Te_Li_001_110_51at_Li_Cmmm_1_Te_R-3m_1,0.054,0.122,-0.109,0.122,0.122
7,Li2Te,mp-2530,Li,"(1, 1, 1)","(1, 1, 0)",3.0,4.0,2.0,5.0,0.012,"(Li_Cmmm_1, Te_R-3m_1)",51,Li2Te_Li_111_110_51at_Li_Cmmm_1_Te_R-3m_1,0.054,0.122,-0.109,0.122,0.122
8,Li2Te,mp-2530,Li,"(0, 0, 1)","(1, 1, 0)",3.0,4.0,2.0,5.0,0.012,"(Li_Cmmm_1, Te_R-3m_1)",51,Li2Te_Li_001_110_51at_Li_Cmmm_1_Te_R-3m_1,0.054,0.122,-0.109,0.122,0.122
9,Li2Te,mp-2530,Li,"(1, 1, 1)","(1, 1, 0)",3.0,4.0,2.0,5.0,0.012,"(Li_Cmmm_1, Te_R-3m_1)",51,Li2Te_Li_111_110_51at_Li_Cmmm_1_Te_R-3m_1,0.054,0.122,-0.109,0.122,0.122


In [8]:
dic_st_final.keys()

dict_keys(['mp-8926', 'mp-568273', 'mp-29463', 'mp-1198740', 'mp-1185319', 'mp-27275', 'mp-755225', 'mp-15960', 'mp-542435', 'mp-3887', 'mp-28549', 'mp-13944', 'mp-22899', 'mp-23703', 'mp-14712', 'mp-985585', 'mp-557964', 'mp-23259', 'mp-2530', 'mp-976280', 'mp-5001', 'mp-1176564', 'mp-942733', 'mp-768960', 'mp-1222669', 'mp-754204', 'mp-1185301', 'mp-1189725', 'mp-28593', 'mp-570935', 'mp-569782'])

In [22]:
from pymatgen.analysis.interfaces.zsl import ZSLGenerator
# from pymatgen.analysis.interfaces.coherent_interfaces import generate_coherent_interface

# Your own substrate slab
st = dic_st_final['mp-3887']
# st = Structure.from_file('LiCoO2.cif')


slabgen = SlabGenerator(initial_structure = st, miller_index = (0,0,1), min_slab_size = 7, 
                            min_vacuum_size = 10, lll_reduce = True, center_slab = False, primitive = True)
slabs = slabgen.get_slabs(symmetrize=True )
print(len(slabs))
substrate_slab = slabs[1]

2


In [23]:
substrate_slab